In [4]:
%pylab inline
import pandas as pd
import seaborn as sns
import numpy as np
import scipy 
import matplotlib.pyplot as plt
import matplotlib.style as style

import sys
from sklearn.model_selection import train_test_split
import pydot
#imports the own created package

from sklearn.tree import export_graphviz

import warnings
warnings.filterwarnings('ignore')


Populating the interactive namespace from numpy and matplotlib


In [10]:
#imports pickle file created and saved in 'load_and_clean.ipynb'
df_both_seasons_home = pd.read_pickle('df_both_seasons_essentials')
df_both_seasons_home.head(20)


,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HST,AST,FTR,HTGDIFF,ATGDIFF
0,9,12,2020,Fulham,Arsenal,0,3,2,6,0,-3,3
1,9,12,2020,Crystal Palace,Southampton,1,0,3,5,2,1,-1
2,9,12,2020,Liverpool,Leeds,4,3,6,3,2,1,-1
3,9,12,2020,West Ham,Newcastle,0,2,3,2,0,-2,2
4,13,9,2020,West Brom,Leicester,0,3,1,7,0,-3,3
5,13,9,2020,Tottenham,Everton,0,1,5,4,0,-1,1
6,14,9,2020,Brighton,Chelsea,1,3,3,5,0,-2,2
7,14,9,2020,Sheffield United,Wolves,0,2,2,4,0,-2,2
8,19,9,2020,Everton,West Brom,5,2,7,4,2,3,-3
9,19,9,2020,Leeds,Fulham,4,3,7,6,2,1,-1


In [11]:
def avg_goal_diff(df, column_name, a_h_team, a_h_goal_letter):
    """
    input: 
        df = dataframe with all results
       column_name = name of the new column
        a_h_team = HomeTeam or AwayTeam
        a_h_goal_letter = 'H' for home or 'A' for away
    output: 
        df: dataframe with a new column (H/A)TGDIFF
       
    """
    df[column_name] = 0
    avg_per_team = {}
    all_teams = df[a_h_team].unique()
    for t in all_teams:
        df_team = df[df[a_h_team]==t]
        result = df_team['{}TGDIFF'.format(a_h_goal_letter)].rolling(10).mean()
        df_team[column_name] = result.shift(-9)
        avg_per_team[t] = df_team
        
    df2 = pd.DataFrame()
    for v in avg_per_team.values():
        df2 = df2.append(v)
    return df2
    
    
def avg_points(df,column_name, h_or_a_team, h_or_a_letter):
    """
    input: 
        df = dataframe with all results
        column_name = name of the new column
        h_or_a_team = HomeTeam or AwayTeam
        a_h_goal_letter = 'H' for home or 'A' for away
    """
    df[column_name] = 0
    avg_points_team = {}
    all_teams = df[h_or_a_team].unique()
    for t in all_teams:
        df_team = df[df[h_or_a_team]==t]
        result = df_team['FTR'].rolling(10).mean()
        df_team[column_name] = result.shift(-9)
        avg_points_team[t] = df_team
    df2 = pd.DataFrame()
    for v in avg_points_team.values():
        df2 = df2.append(v)
    return df2
    

In [12]:
df_both_seasons=avg_goal_diff(df_both_seasons_home, 'AVGHTGDIFF', 'HomeTeam', 'H')
df_both_seasons.head()

,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HST,AST,FTR,HTGDIFF,ATGDIFF,AVGHTGDIFF
0,9,12,2020,Fulham,Arsenal,0,3,2,6,0,-3,3,0.1
26,28,9,2020,Fulham,Aston Villa,0,3,3,5,0,-3,3,0.4
50,24,10,2020,Fulham,Crystal Palace,1,2,3,10,0,-1,1,0.8
66,11,2,2020,Fulham,West Brom,2,0,6,1,2,2,-2,0.6
82,22,11,2020,Fulham,Everton,2,3,6,7,0,-1,1,0.5


In [13]:
df_both_seasons=df_both_seasons.sort_values(['Year', 'Month','Day'], ascending=False)

In [14]:
df_both_seasons=avg_points(df_both_seasons, 'AVGP', 'HomeTeam', 'H')


In [15]:
df_both_seasons.head()

,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HST,AST,FTR,HTGDIFF,ATGDIFF,AVGHTGDIFF,AVGP
0,9,12,2020,Fulham,Arsenal,0,3,2,6,0,-3,3,0.1,1.0
82,22,11,2020,Fulham,Everton,2,3,6,7,0,-1,1,0.5,1.0
50,24,10,2020,Fulham,Crystal Palace,1,2,3,10,0,-1,1,0.8,1.2
505,7,10,2020,Fulham,Cardiff,2,0,7,1,2,2,-2,NaN,1.4
26,28,9,2020,Fulham,Aston Villa,0,3,3,5,0,-3,3,0.4,1.4


In [16]:
df_both_seasons=df_both_seasons.sort_values(['Year', 'Month','Day'], ascending=False)

In [17]:
df_both_seasons.columns

Index(['Day', 'Month', 'Year', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HST',
       'AST', 'FTR', 'HTGDIFF', 'ATGDIFF', 'AVGHTGDIFF', 'AVGP'],
      dtype='object')

In [69]:
df_both_seasons.head()

,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HST,AST,HTR,HTGDIFF,ATGDIFF,AVGHTGDIFF,AVGP
0,9,12,2020,Fulham,Arsenal,0,3,2,6,0,-3,3,0.1,0.8
1,9,12,2020,Crystal Palace,Southampton,1,0,3,5,2,1,-1,0.2,1.2
2,9,12,2020,Liverpool,Leeds,4,3,6,3,2,1,-1,2.0,1.8
3,9,12,2020,West Ham,Newcastle,0,2,3,2,1,-2,2,0.0,1.4
347,7,12,2020,Tottenham,Arsenal,2,1,9,4,1,1,-1,NaN,NaN


In [18]:
df_both_seasons["AVGP"].fillna(value=df_both_seasons["AVGP"].mean(),inplace=True)
df_both_seasons["AVGHTGDIFF"].fillna(value=df_both_seasons["AVGHTGDIFF"].mean(),inplace=True)

In [19]:
df_both_seasons.head(20)

,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HST,AST,FTR,HTGDIFF,ATGDIFF,AVGHTGDIFF,AVGP
0,9,12,2020,Fulham,Arsenal,0,3,2,6,0,-3,3,0.100000,1.000000
1,9,12,2020,Crystal Palace,Southampton,1,0,3,5,2,1,-1,0.200000,0.900000
2,9,12,2020,Liverpool,Leeds,4,3,6,3,2,1,-1,2.000000,1.900000
3,9,12,2020,West Ham,Newcastle,0,2,3,2,0,-2,2,0.000000,1.000000
347,7,12,2020,Tottenham,Arsenal,2,1,9,4,2,1,-1,0.317771,1.300000
346,7,12,2020,Aston Villa,Crystal Palace,2,0,9,5,2,2,-2,0.317771,0.700000
345,7,12,2020,Wolves,Everton,3,0,7,2,2,3,-3,0.317771,1.400000
348,7,12,2020,Bournemouth,Leicester,4,1,4,5,2,3,-3,0.317771,0.900000
382,2,12,2020,Reading,West Brom,1,2,1,7,0,-1,1,0.317771,1.168675
381,2,12,2020,Millwall,Fulham,1,1,2,2,1,0,0,0.317771,1.168675


In [72]:
df_both_seasons.shape


(579, 14)

In [20]:
##ON sauvegarde nos données après prétraitements
import pickle
df_both_seasons.to_pickle('df_both_seasons_with_features')